# データの取得、整形

In [1]:
import sys
sys.path

['d:\\DEV\\competition\\text\\signate_classfication_medical_papers',
 'd:\\masa\\Anaconda3\\python38.zip',
 'd:\\masa\\Anaconda3\\DLLs',
 'd:\\masa\\Anaconda3\\lib',
 'd:\\masa\\Anaconda3',
 '',
 'C:\\Users\\masa\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'd:\\masa\\Anaconda3\\lib\\site-packages',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\geoplot-0.4.1-py3.8.egg',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\contextily-1.0.0-py3.8.egg',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\mapclassify-2.3.0-py3.8.egg',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\descartes-1.1.0-py3.8.egg',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\win32',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'd:\\masa\\Anaconda3\\lib\\site-packages\\Pythonwin']

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("train.csv", index_col=0)

In [4]:
df["abstract"]=df["abstract"].where(df["abstract"].notna(),df["title"])

In [5]:
df.head()

,title,abstract,judgement
id,,,
0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0
1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0
2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0
3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0
4,Prolonged shedding of SARS-CoV-2 in an elderly...,Prolonged shedding of SARS-CoV-2 in an elderly...,0


In [6]:
len(df)

27145

## インストール

In [7]:
#!pip install ipython -U

In [8]:
# ライブラリのインストール
#!python -m pip install --upgrade pip
#!pip install -U fugashi ipadic \
#    transformers lime captum\
#    scikit-learn numpy --user


# データローラーなど

In [9]:
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, df):
        self.features = [
            {
                'abstract': row.abstract,
                'judgement': row.judgement
            } for row in tqdm(df.itertuples(), total=df.shape[0])
        ]
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]

train_df, eval_df = train_test_split(df, train_size=0.7)
eval_df, test_df = train_test_split(eval_df, train_size=0.5)

train_dataset = CustomDataset(train_df)
eval_dataset = CustomDataset(eval_df)
test_dataset = CustomDataset(test_df)

100%|██████████| 4072/4072 [00:00<00:00, 811363.70it/s]


In [10]:
import torch
from transformers import AutoTokenizer

class CustomCollator():
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __call__(self, examples):
        examples = {
            'abstract': list(map(lambda x: x['abstract'], examples)),
            'judgement': list(map(lambda x: x['judgement'], examples))
        }
        
        encodings = self.tokenizer(examples['abstract'],
                                   padding=True,
                                   truncation=True,
                                   max_length=self.max_length,
                                   return_tensors='pt')
        encodings['judgement'] = torch.tensor(examples['judgement'])
        return encodings
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
custom_collator = CustomCollator(tokenizer)

In [11]:
# ライブラリのインポート
import os
import re
import sys

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients
from lime.lime_text import LimeTextExplainer
import numpy as np
import torch
from torch.nn import functional as F
import transformers
from sklearn.model_selection import train_test_split

In [12]:
# BERTモデルの動作デバイス
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
import torch.nn as nn
from transformers import AutoModel
from transformers.modeling_outputs import ModelOutput

class CustomNet(nn.Module):
    def __init__(self, pretrained_model, num_categories, loss_function=None):
        super().__init__()
        self.bert = pretrained_model
        self.hidden_size = self.bert.config.hidden_size
        self.linear = nn.Linear(self.hidden_size, num_categories)
        self.loss_function = loss_function
    
    def forward(self,
                input_ids,
                attention_mask=None,
                position_ids=None,
                token_type_ids=None,
                output_attentions=False,
                output_hidden_states=False,
                judgement=None):
        
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            position_ids=position_ids,
                            token_type_ids=token_type_ids,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states)
        
        state = outputs.last_hidden_state[:, 0, :]
        state = self.linear(state)
        
        loss=None
        if judgement is not None and self.loss_function is not None:
            loss = self.loss_function(state, judgement)
        
        attentions=None
        if output_attentions:
            attentions=outputs.attentions
        
        hidden_states=None
        if output_hidden_states:
            hidden_states=outputs.hidden_states
        
        return ModelOutput(
            logits=state,
            loss=loss,
            last_hidden_state=outputs.last_hidden_state,
            attentions=attentions,
            hidden_states=hidden_states
        )

categories = [0,1]
loss_fct = nn.CrossEntropyLoss()
pretrained_model = AutoModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model = CustomNet(pretrained_model, len(categories), loss_fct)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
model = model.to(device)

# Fine tunining

In [15]:
optimizer = transformers.AdamW(model.parameters(), lr=1e-5)

C:\Users\masa\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
from transformers import EvalPrediction
from typing import Dict
from sklearn.metrics import precision_score, recall_score, f1_score

def custom_compute_metrics(res: EvalPrediction) -> Dict:
    # res.predictions, res.label_idsはnumpyのarray
    pred = res.predictions.argmax(axis=1)
    target = res.label_ids
    precision = precision_score(target, pred, average='macro')
    recall = recall_score(target, pred, average='macro')
    f1 = f1_score(target, pred, average='macro')
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./output/model',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    label_names=['judgement'],
    lr_scheduler_type='constant',
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    remove_unused_columns=False
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
from transformers import Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=custom_collator,
    compute_metrics=custom_compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])

C:\Users\masa\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19001
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 237600


  0%|          | 0/237600 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 4072
  Batch size = 8


{'loss': 0.1261, 'learning_rate': 5e-05, 'epoch': 1.0}


  0%|          | 0/509 [00:00<?, ?it/s]

...

  0%|          | 0/509 [00:00<?, ?it/s]

d:\masa\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./output/model\checkpoint-7128
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 0.14664702117443085, 'eval_precision': 0.4867387033398821, 'eval_recall': 0.5, 'eval_f1': 0.49328023892483824, 'eval_runtime': 73.5337, 'eval_samples_per_second': 55.376, 'eval_steps_per_second': 6.922, 'epoch': 3.0}


tokenizer config file saved in ./output/model\checkpoint-7128\tokenizer_config.json
Special tokens file saved in ./output/model\checkpoint-7128\special_tokens_map.json


ValueError: 'output\\model\\checkpoint-2376' is not in list

In [ ]:
trainer.__dict__

{'args': TrainingArguments(
 _n_gpu=1,
 adafactor=False,
 adam_beta1=0.9,
 adam_beta2=0.999,
 adam_epsilon=1e-08,
 auto_find_batch_size=False,
 bf16=False,
 bf16_full_eval=False,
 data_seed=None,
 dataloader_drop_last=False,
 dataloader_num_workers=0,
 dataloader_pin_memory=True,
 ddp_bucket_cap_mb=None,
 ddp_find_unused_parameters=None,
 ddp_timeout=1800,
 debug=[],
 deepspeed=None,
 disable_tqdm=False,
 do_eval=True,
 do_predict=False,
 do_train=False,
 eval_accumulation_steps=None,
 eval_delay=0,
 eval_steps=None,
 evaluation_strategy=epoch,
 fp16=False,
 fp16_backend=auto,
 fp16_full_eval=False,
 fp16_opt_level=O1,
 fsdp=[],
 fsdp_min_num_params=0,
 fsdp_transformer_layer_cls_to_wrap=None,
 full_determinism=False,
 gradient_accumulation_steps=1,
 gradient_checkpointing=False,
 greater_is_better=True,
 group_by_length=False,
 half_precision_backend=auto,
 hub_model_id=None,
 hub_private_repo=False,
 hub_strategy=every_save,
 hub_token=<HUB_TOKEN>,
 ignore_data_skip=False,
 include_i

In [ ]:
import pandas as pd
import os

df = pd.read_pickle('./input/livedoor_data.pickle')
# カテゴリーのID列を付与しておく
categories = df['category'].unique().tolist()
category2id = {cat: categories.index(cat) for cat in categories}
df['category_id'] = df['category'].map(lambda x: category2id[x])

df.sample(3)